# Forest and Natural Resources Management Division Quarterly Accomplishments Script
### This script is broken into 2 major parts: Urban and Community Forestry Stats and FNRM Accomplishments. Part 1 is used to find the total acres of and total number of communities assisted with UC&F. Part 2 is used to create the dot density map showing the quarterly accomplishments. More detailed information can be found in the README on GitHub. https://github.com/jgorman-tfs/FRD-Accomplishments

### Side note for future readers - The division changed its name from FRD to FNRM

## Variables and Imports

In [100]:
import pandas as pd
import os
#Quarter and fiscal year
qtr = "FY2026Q1"
#Set folder path to new quarter folder
folder_path = rf'D:\ArcGIS_Projects\FRDAccomplishments\{qtr}'
#This shapefile is provided in sharepoint. If your using your own, you MUST calculate a new field with the County names of each city.
cities = "Texas_Places_WithCounties"
gdb = r"D:\ArcGIS_Projects\FRDAccomplishments\FRDAccomplishments.gdb"
accomp_from_access_path = r"D:\ArcGIS_Projects\FRDAccomplishments\FY2026Q1\Accomplishments.XLSX"
ucf_spreadsheet_path = r"D:\ArcGIS_Projects\FRDAccomplishments\FY2026Q1\Q1FY26_Urban_OG_TESTING.xlsx"
spam_sheet = "spam_raw"
elmr_sheet = "elmr_raw"

con_ed_activity_list = ["Arbor Day Program", 
                        "UF Presentation", 
                        "Brochure/Web/Newsletter/Media", 
                        "Conference/Workshop/Training", 
                        "Education/Outreach Event or Presentation",
                        "UF Training Given",
                        "Arbor Day/Tree City USA Event"
                       ]
ta_activity_list = ["Tree Planting Event",
                    "Tree Board or Group Activities",
                    "UF Incidental Assist",
                    "UF Individual Assist",
                    "EAB and other Pest Detection/Planning",
                    "Landscape Plan or Site-Specific Issue",
                    "Management Plan",
                    "Tree Inventory or Assessment",
                    "Tree Planting/Maintenance Program",
                    "Tree Ordinance/Policy"
                   ]

## Part 1 : Urban and Community Forestry Stats

### <u><b>Before you begin:<b><u>
### 1. Combine the SPAM and ELMR data into one workbook (xlxs). 
### 2. Make 2 sheets: <u><b> spam_raw and elmr_raw <b><u>
### 3. Copy the city and activity name from the spam table into the spam_raw sheet 
### 4. Copy the office, county, and activity name from the ELMR table to elmr_raw. Change abbreviated offices to the full name (i.e KER -> Kerville

### SPAM SHEET COLUMNS : City , Activity Name
### ELMR SHEET COLUMNS: City, Activity Name, County

### Read the sheets into a single pandas df. Make sure the counts match up.

In [ ]:
dfs = pd.read_excel(
    ucf_spreadsheet_path,
    sheet_name=["spam_raw", "elmr_raw"],
    usecols=["City", "Activity Name"]
)
sheet_counts = {sheet: len(df) for sheet, df in dfs.items()}
    
df = (
    pd.concat(dfs.values(), ignore_index=True)
      .rename(columns={
          "City": "city",
          "Activity Name": "activity_name"
      })
)

print(df.head())
# Count combined rows
combined_count = len(df)
expected_count = sum(sheet_counts.values())

print(f"\nExpected total rows: {expected_count}")
print(f"Actual combined rows: {combined_count}")

assert combined_count == expected_count, "Row count mismatch!"
print("Row counts match")

### Get the unique activity names

In [ ]:
unique_activities = (
    df[["activity_name"]]
    .drop_duplicates()
    .sort_values("activity_name")
    .reset_index(drop=True)
)
print(unique_activities)

### Map the activity type to the activity name. 

### Check to see which activities are not included. You should see General Public/Home/Phone Consultation and Informational Meeting Attended. It was decided that these are not to be included for now, however other categories may appear. Either use best judgement on where to include them or drop them or ask Gretchen/Melissa/Michelle.

In [ ]:
df["activity_type"] = None

df.loc[
    df["activity_name"].isin(con_ed_activity_list),
    "activity_type"
] = "Conservation Education"

df.loc[
    df["activity_name"].isin(ta_activity_list),
    "activity_type"
] = "Technical Assistance"

   
unmapped_counts = (
    df[df["activity_type"].isna()]["activity_name"]
    .value_counts()
    .reset_index()
)

unmapped_counts.columns = ["activity_name", "count"]
print(unmapped_counts)

total_unmapped = unmapped_counts["count"].sum()
print("Total of activities to be dropped: ", total_unmapped)

### Check to make sure the number of rows dropped is expected. This should be the total of unmapped rows minus the total rows of the original data frame.

In [ ]:
df_filtered = df.dropna(subset=["activity_type"]).reset_index(drop=True)
assert combined_count - total_unmapped == len(df_filtered), "Counts do not line up. The incorrect number of rows were dropped"
print("Counts are as expected")


### Get the Texas_PLaces_WithCounties layer as a dataframe

In [105]:
fields = ["CityName", "Acres","CountyName"]

# Convert to NumPy array
arr = arcpy.da.TableToNumPyArray(cities, fields)
feature_layer_df = pd.DataFrame(arr)

print(feature_layer_df.head())

     CityName        Acres CountyName
0  Kingsville  8900.015625    Kleberg
1      Gorman  1051.673950   Eastland
2    Comanche  2918.978027   Comanche
3     Gustine   585.835266   Comanche
4    Channing   634.972778    Hartley


### Normalize city names to remove spaces and merge the city list with the Places layer

In [ ]:
df_filtered["city"] = df_filtered["city"].str.replace(" ", "", regex=False)

df_merged = df_filtered.merge(
    feature_layer_df,
    left_on="city",      # column in df_filtered
    right_on="CityName", # column in feature_layer_df
    how="left"
)
df_merged = df_merged.drop(columns=["CityName"])
print(df_merged.head())

### Print out the cities that did not get matched. 

### It is expected to have a few because sometimes SPAM reports the county instead of the city. In that case, you can drop any Texas rows and rows that have counties instead of cities (i.e. Tarrant). There may also be misspellings. In that case you can adjust the spelling on the spreadsheet and run the cells again. 

In [ ]:
unmatched_cities = (
    df_merged[df_merged["Acres"].isna()]["city"]
    .value_counts()
    .reset_index()
)

unmatched_cities.columns = ["city", "count"]

print(unmatched_cities)

### Drop the unmatched cities to get a final dataframes; one for the full city list with duplicates (minus unmatched cities and activities) and one of the unique cities. Calculate the total acres assisted for unique cities.

In [ ]:
ucf_df_final = df_merged.dropna(subset=["Acres"]).reset_index(drop=True) 
unique_cities = (
    ucf_df_final[["city","Acres"]]
    .drop_duplicates()
    .sort_values("city")
    .reset_index(drop=True)
)
print(ucf_df_final.head())
print("\nTotal Cities with Duplicates: ", len(ucf_df_final))
print("\nTotal Unique Cities :", len(unique_cities))
total_acres = unique_cities["Acres"].sum()
print("\nTotal Acres Assisted: ", total_acres)

### Send the final dataframes to a single spreadsheet with a sheet for each df

In [ ]:
city_output_filename = f"{qtr}_CityActivities.xlsx"

# Full path to the Excel file
city_output_xlsx = os.path.join(folder_path, city_output_filename)

# Write the Excel file with two sheets
with pd.ExcelWriter(city_output_xlsx, engine="openpyxl") as writer:
    ucf_df_final.to_excel(writer, sheet_name="FinalCityListFull", index=False)
    unique_cities.to_excel(writer, sheet_name="UniqueCities", index=False)

print(f"Excel file saved to: {city_output_xlsx}")

## Part 2: Accomplishments Map

In [ ]:
#Create a copy of the County Layer that will be used for the map
counties_new = f"{qtr}_Counties"
arcpy.FeatureClassToFeatureClass_conversion(f"{gdb}\\{counties_template}", gdb, counties_new)

In [106]:
spam_df = pd.read_excel(ucf_spreadsheet_path, sheet_name=spam_sheet)
spam_df["City"] = df_filtered["city"].str.replace(" ", "", regex=False)
spam_df.head()

,City,Activity Name
0,Austin,Brochure/Web/Newsletter/Media
1,Dallas,Brochure/Web/Newsletter/Media
2,Victoria,Landscape Plan or Site-Specific Issue
3,McAllen,Tree Planting/Maintenance Program
4,GrandPrairie,Tree Planting/Maintenance Program


In [117]:
spam_df_merged = spam_df.merge(
    feature_layer_df[["CityName", "CountyName"]],  # only keep relevant columns
    left_on="City",      # column in spam_df
    right_on="CityName", # column in feature_layer_df
    how="left"
)
unmatched_counties = (
    spam_df_merged[spam_df_merged["CountyName"].isna()]["City"]
    .value_counts()
    .reset_index()
)

unmatched_counties.columns = ["City", "count"]

print(unmatched_counties)

         City  count
0       Texas     21
1    McLennan      6
2     Johnson      2
3   Limestone      2
4     Tarrant      1
5        Jack      1
6       Falls      1
7  Williamson      1


In [119]:
import numpy as np

spam_df_merged["CountyName"] = np.where(
    spam_df_merged["CountyName"].isna(),  # only where CountyName is NaN
    np.where(
        spam_df_merged["City"] == "Texas",  # if city is "Texas"
        "Travis",                           # use "Travis"
        spam_df_merged["City"]              # else use city name
    ),
    spam_df_merged["CountyName"]            # if CountyName is not NaN, keep it
)
unmatched_counties = (
    spam_df_merged[spam_df_merged["CountyName"].isna()]["City"]
    .value_counts()
    .reset_index()
)

unmatched_counties.columns = ["City", "count"]

print(unmatched_counties)

Empty DataFrame
Columns: [City, count]
Index: []


,City,Activity Name,CityName,CountyName
0,Austin,Brochure/Web/Newsletter/Media,Austin,Travis
1,Dallas,Brochure/Web/Newsletter/Media,Dallas,Dallas
2,Victoria,Landscape Plan or Site-Specific Issue,Victoria,Victoria
3,McAllen,Tree Planting/Maintenance Program,McAllen,Hidalgo
4,GrandPrairie,Tree Planting/Maintenance Program,GrandPrairie,Dallas
